In [2]:
import glob
import os;
import six.moves.cPickle as pickle
import gzip
import numpy as np
import PIL.Image as Im
import keras
import theano
import matplotlib.pyplot as plt
# reading dataset

mscoco= r"C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting"
split="train2014"
path_to_dataset= os.path.join(mscoco, split)

cap= "dict_key_imgID_value_caps_train_and_valid.pkl"
path_to_captions= os.path.join(mscoco, cap)

images= glob.glob(path_to_dataset+"/*jpg")
theano.config.floatX = 'float32'


# Getting a minibatch of data
def minibatch_of_data(batch_index,batch_size,mscoco= r"C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting", split="train2014"):
    im_batch=images[batch_size*batch_index: batch_index+1*batch_size]
    b_input=[]
    b_target=[]
    im_arr=[]
    for i, path in enumerate(im_batch):
        Image=Im.open(path)
        im_arr = np.array(Image)
        
        # get the central part of image and set all values in that part to 0
        
        centre_of_image= (int(np.floor(im_arr.shape[0] / 2.)), int(np.floor(im_arr.shape[1] / 2.)))
        
        # setting central part of image to 0
        
        if len(im_arr==3):
            # ignoring gray scale images
            input= np.copy(im_arr)
            # editing centre on image
            input[centre_of_image[0]-16:centre_of_image[0]+16,centre_of_image[1]-16:centre_of_image[1]+16]=0
            target= im_arr[centre_of_image[0]-16:centre_of_image[0]+16,centre_of_image[1]-16:centre_of_image[1]+16]
            b_input.append(np.array(input).transpose(2,0,1).reshape(3,64,64))
            b_target.append(np.array(target).transpose(2,0,1).reshape(3,32,32))
            # normalized data
        return np.array(b_input)/255, np.array(b_target)/255
        
def datasetsz(mscoco=r"C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting", split="train2014"):
    data_path = os.path.join(mscoco, split)
    return len(glob.glob(data_path + "/*.jpg"))
        
def loaddata(mscoco=r"C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting"):
    X_train, y_train= minibatch_of_data(0, datasetsz(mscoco= r"C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting", split="train2014"))
    X_val, y_val= minibatch_of_data(0, datasetsz(mscoco=r"C:\Users\NAVPREET\AnacondaProjects\inpainting\inpainting", split="val2014"))
    return X_train, y_train,X_val, y_val



def saveImages(images, path='../results'):
    for i, img in enumerate(images):
        save_dir = path + '/result_'+str(i)+'.jpg'

        img = Im.fromarray(img.astype('uint8'))
        img.save(save_dir)

X_train, y_train, X_val, y_val=loaddata()
print(X_train.shape)


def getimage(inner, outer):
    
    #getting images back in original scale
    inner, outer=inner*255, outer*255
    outer= outer.tanspose(0,2,3,1).reshape(len(outer), 64, 64, 3)
    iner=inner.transpose(0, 2, 3, 1). reshape(len(inner), 32, 32, 3)
    
    Im_batch=[]
    for i, img in enumerate(outer):
        centre= (int(np.floor(outer.shape[1]/2.)), int(np.floor(outer.shape[2]/2.)))
        rebuilt_image= np.copy(img)
        rebuilt_image[centre[0]-16: centre[0]+16, centre[1]-16: centre[1]+16]=inner[1,:,:,:]
        Im_batch.img.append(rebuilt_image)
    return np.array(Im_batch)



def draw_image(input, target, pred):
    """ Draws the true image and the predicted image """
    # dimshuffle to put channels in last dim so we can plot (x, y, channel)
    input = input.transpose((1, 2, 0))
    target = target.transpose((1, 2, 0))
    pred = pred.transpose((1, 2, 0))

    print(input.shape)
    # Place the center back of the true image
    center = (int(np.floor(input.shape[0] / 2.)), int(np.floor(input.shape[1] / 2.)))

    print(center)
    input[center[0] - 16:center[0] + 16, center[1] - 16:center[1] + 16, :] = target

    # Draw the target
    plt.figure(figsize=(8,8)).canvas.set_window_title("Target")
    plt.imshow(input)

    # Place the center back of the predicted image
    # Draw the prediction
    input[center[0] - 16:center[0] + 16, center[1] - 16:center[1] + 16, :] = pred
    plt.figure(figsize=(8,8)).canvas.set_window_title("Prediction")
    plt.imshow(input)

    plt.show()
    return






Using Theano backend.


(1L, 3L, 64L, 64L)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Dropout, MaxPooling2D, Deconvolution2D, UpSampling2D, Reshape, Input
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.models import load_model, Model
import scipy as sp

# defining hyperparameters of model
nb_epoch=10
CPname=None

for j in range(nb_epoch,-1,-1):
    epochStr = str(j)
    #runStr = str(i)

    if len(epochStr)==1:
        epochStr=str(0)+epochStr

    if os.path.exists('CP/cnn/cnn-'+epochStr+'.h5'):
        CPName='CP/cnn/cnn-'+epochStr+'.h5'
        epoch=j+1
        #run=i+1
        found = True
        break
#if found:
    #break
        
        
        
        
model=Sequential()
nb_epoch=1
batch_size=128
nb_filters=10
nb_pool=2
nb_conv=2
sgd=SGD(lr=0.2, decay=1e-6, momentum=0.9, nesterov=True)


# adding layers to model
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', input_shape=(3, 64, 64), activation='relu', strides=(1,1), data_format="channels_first"))
model.add(Activation('relu'))
model.add(Convolution2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', strides=(1,1), data_format="channels_first"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', strides=(1,1), data_format="channels_first"))
model.add(Activation('relu'))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
model.add(Activation('relu'))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(6912))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(3072))
model.add(Activation('relu'))


model.add(Reshape([3,32, 32]))
#model.add(UpSampling2D([2, 2], data_format="channels_first"))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
model.add(Convolution2D(filters=18, kernel_size=(5,5), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
model.add(Convolution2D(filters=16, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
#model.add(UpSampling2D([2,2]))
model.add(Convolution2D(filters=3, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu', data_format="channels_first"))
 #do all calculations do see size of all inputs and outputs
model.add(Reshape([3, 32, 32]))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
history= model.fit(X_train, y_train, batch_size=10, epochs=1, verbose=0, validation_data=(X_val, y_val))
#model.save(cnn.h5)
#print(history.history.keys())
#plt.plot(history.history['acc'])
#plt.plot(history.history['val_acc'])
#plt.title('Accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper_left')

score=model.evaluate(X_val, y_val, verbose=0)
print('Test Score:', score[0] )
print('accuracy:', score[1])
plt.show()

ex=1

ex_x_train = X_train[ ex *batch_size:( ex +1 ) *batch_size]
ex_y_train = y_train[ ex *batch_size:( ex +1 ) *batch_size]
ex_x_valid = X_valid[ ex *batch_size:( ex +1 ) *batch_size]
ex_y_valid = y_valid[ ex *batch_size:( ex +1 ) *batch_size]
    
pred = model.predict(ex_x_train, ex_y_train)
predv = model.predict(ex_x_valid, ex_y_valid)
pred = squeeze(pred)
predv = squeeze(predv)

i=2
draw_image(ex_x_train[i], ex_y_train[i], pred[i])
draw_image(ex_x_valid[i], ex_y_valid[i], predv[i])

plt.show()